# TCGA-BRCA Analysis: *Cancer Subtype* classification

### Dataset Source

- **Omics Data**: [FireHose BRCA](http://firebrowse.org/?cohort=BRCA)
- **Clinical and PAM50 Data**: [TCGAbiolinks](http://bioconductor.org/packages/release/bioc/html/TCGAbiolinks.html)

##### Dataset Overview

- **Original Data**:

    - **Methylation**: 20,107 × 885
    - **mRNA**: 18,321 × 1,212
    - **miRNA**: 503 × 1,189
    - **PAM50**: 1,087 × 1
    - **Clinical**: 1,098 × 101

- **PAM50 Subtype Counts**:

    - **LumA**: 419
    - **LumB**: 140
    - **Basal**: 130
    - **Her2**: 46
    - **Normal**: 34

### Patients in Every Dataset

- Total patients present in methylation, mRNA, miRNA, PAM50, and clinical: **769**

### Final Shapes (Per-Patient)

After aggregating multiple aliquots by mean, all modalities align on 769 patients:

- **Methylation**: 769 × 20,106
- **mRNA**: 769 × 20,531
- **miRNA**: 769 × 503
- **PAM50**: 769 × 1
- **Clinical**: 769 × 119

### Data Summary Table

| Stage                          | Clinical    | Methylation  | miRNA       | mRNA           | PAM50 (Subtype Counts)                                         | Notes                                   |
| ------------------------------ | ----------- | ------------ | ----------- | -------------- | -------------------------------------------------------------- | --------------------------------------- |
| **Original Raw Data**          | 1,098 × 101 | 20,107 × 885 | 503 × 1,189 | 18,321 × 1,212 | LumA: 509<br>LumB: 209<br>Basal: 192<br>Her2: 82<br>Normal: 40 | Raw FireHose & TCGAbiolinks files       |
| **Patient-Level Intersection** | 769 × 101   | 769 × 20,107 | 769 × 1,046 | 769 × 20,531   | LumA: 419<br>LumB: 140<br>Basal: 130<br>Her2: 46<br>Normal: 34 | Patients with complete data in all sets |

### Reference Links

- [FireHose BRCA](http://firebrowse.org/?cohort=BRCA)
- [TCGAbiolinks](http://bioconductor.org/packages/release/bioc/html/TCGAbiolinks.html)
- [Direct Download BRCA](http://firebrowse.org/?cohort=BRCA&download_dialog=true)


In [ ]:
import pandas as pd
from pathlib import Path
root = Path("/home/vicente/Github/BioNeuralNet/TCGA_BRCA_DATA")

mirna_raw = pd.read_csv(root/"BRCA.miRseq_RPKM_log2.txt", sep="\t",index_col=0,low_memory=False)                            
rna_raw = pd.read_csv(root / "BRCA.uncv2.mRNAseq_RSEM_normalized_log2.txt", sep="\t",index_col=0,low_memory=False)
meth_raw = pd.read_csv(root/"BRCA.meth.by_mean.data.txt", sep='\t',index_col=0,low_memory=False)
clinical_raw = pd.read_csv(root / "BRCA.clin.merged.picked.txt",sep="\t", index_col=0, low_memory=False)

# display all shapes and first few rows of each dataset
display(mirna_raw.iloc[:3,:5])
display(mirna_raw.shape)

display(rna_raw.iloc[:3,:5])
display(rna_raw.shape)

display(meth_raw.iloc[:3,:5])
display(meth_raw.shape)

display(clinical_raw.iloc[:3,:5])
display(clinical_raw.shape)

## TCGA-BioLink: Pam50

This section demonstrates how to use the `TCGAbiolinks` R package to access and download clinical and molecular subtype data. It begins by ensuring `TCGAbiolinks` is installed, then loads the package. It retrieves PAM50 molecular subtype labels using `TCGAquery_subtype()` and writes them to a CSV file. Additionally, it downloads clinical data using `GDCquery_clinic()` and formats it with `GDCprepare_clinic()`, saving the result as another CSV file.

```R
  # Install TCGAbiolinks
  if (!requireNamespace("TCGAbiolinks", quietly = TRUE)) {
    if (!requireNamespace("BiocManager", quietly = TRUE))
      install.packages("BiocManager")
    BiocManager::install("TCGAbiolinks")
  }

  # Load the library
  library(TCGAbiolinks)

  # Download PAM50 subtype labels
  pam50_df <- TCGAquery_subtype(tumor = "BRCA")[ , c("patient", "BRCA_Subtype_PAM50")]
  write.csv(pam50_df, file = "BRCA_PAM50_labels.csv", row.names = FALSE, quote = FALSE)

  # Download clinical data
  clin_raw <- GDCquery_clinic(project = "TCGA-BRCA", type = "clinical")
  clin_df <- GDCprepare_clinic(clin_raw, clinical.info = "patient")
  write.csv(clin_df, file = "BRCA_clinical_data.csv", row.names = FALSE, quote = FALSE)
```

In [ ]:
pam50 = pd.read_csv(root /"BRCA_PAM50_labels.csv",index_col=0)
clinical_biolinks = pd.read_csv(root /"BRCA_clinical_data.csv",index_col=1)

display(pam50.iloc[:5,:5])
display(pam50.shape)
display(clinical_biolinks.iloc[:5,:5])
display(clinical_biolinks.shape)

## Data Processing Summary

- **Transpose Data**: All raw data (miRNA, RNA, etc.) is flipped so rows represent patients and columns represent features.
- **Standardize Patient IDs**: Patient IDs in all tables are cleaned to the 12-character TCGA format (e.g., `TCGA-AB-1234`) for matching.
- **Handle Duplicates**: Duplicate patient rows are averaged in the omics data. The first entry is kept for duplicate patients in the clinical data.
- **Impute Missing Values (KNN)**: Missing data (NaNs) in the omics datasets are estimated and filled using **K-Nearest Neighbors (KNN)** imputation.
- **Find Common Patients**: The script identifies the largest common cohort of patients that exist in all datasets.
- **Subset Data**: All data tables are filtered down to only this common list of patients, ensuring perfect alignment.
- **Extract Target**: The **PAM50 subtype** column is pulled from the corresponding data table to be used as the final prediction target (y-variable).

In [ ]:
import bioneuralnet as bnn

meth = meth_raw.T
rna = rna_raw.T
mirna = mirna_raw.T
clinical_firehose = clinical_raw.T

print(f"miRNA (samples, features): {mirna.shape}")
print(f"RNA (samples, features): {rna.shape}")
print(f"Methylation (samples, features): {meth.shape}")
print(f"Clinical (samples, features): {clinical_firehose.shape}")

def trim_barcode(idx):
    return idx.to_series().str.slice(0, 12)

# standardize patient IDs across all files
meth.index = trim_barcode(meth.index)
rna.index = trim_barcode(rna.index)
mirna.index = trim_barcode(mirna.index)
clinical_firehose.index = clinical_firehose.index.str.upper()
clinical_firehose.index.name = "Patient_ID"

meth = meth.apply(pd.to_numeric, errors='coerce')
meth = meth.drop(columns=["Composite Element REF"], errors="ignore")

rna = rna.apply(pd.to_numeric, errors='coerce')
mirna = mirna.apply(pd.to_numeric, errors='coerce')

meth = meth.groupby(meth.index).mean()
rna = rna.groupby(rna.index).mean()
mirna = mirna.groupby(mirna.index).mean()

clinical = clinical_firehose[~clinical_firehose.index.duplicated(keep='first')]

for df in [meth, rna, mirna]:
    df.columns = df.columns.str.replace(r"\?", "unknown_", regex=True)
    df.columns = df.columns.str.replace(r"\|", "_", regex=True)
    df.columns = df.columns.str.replace("-", "_", regex=False)
    df.columns = df.columns.str.replace(r"_+", "_", regex=True)
    df.columns = df.columns.str.strip("_")

print(f"\nMethylation shape: {meth.shape}")
print(f"RNA shape: {rna.shape}")
print(f"miRNA shape: {mirna.shape}")
print(f"Clinical shape: {clinical.shape}")

meth.columns = pd.Index(meth.columns.tolist())
rna.columns = pd.Index(rna.columns.tolist())
mirna.columns = pd.Index(mirna.columns.tolist())

# impute missing values using KNN on the finalized omics data
meth = bnn.utils.data.impute_omics_knn(meth, n_neighbors=5)
rna = bnn.utils.data.impute_omics_knn(rna, n_neighbors=5)
mirna = bnn.utils.data.impute_omics_knn(mirna, n_neighbors=5)

# handle duplicate patient IDs in clinical data (keeping first occurrence)
clinical_biolinks = clinical_biolinks[~clinical_biolinks.index.duplicated(keep='first')]
clinical_firehose = clinical_firehose[~clinical_firehose.index.duplicated(keep='first')]

# patients common to both clinical sources and merge
common_clinical_patients = clinical_biolinks.index.intersection(clinical_firehose.index)
clinical_biolinks = clinical_biolinks.loc[common_clinical_patients]
clinical_firehose = clinical_firehose.loc[common_clinical_patients]
clinical = pd.concat([clinical_biolinks, clinical_firehose], axis=1)
clinical.index.name = "Patient_ID"


# final list of patients present in ALL datasets
common_patients = sorted(
    set(meth.index) & 
    set(rna.index) & 
    set(mirna.index) & 
    set(pam50.index) & 
    set(clinical.index)
)

print(f"\nFound: {len(common_patients)} patients across all data types.")

meth = meth.loc[common_patients]
rna = rna.loc[common_patients]
mirna = mirna.loc[common_patients]
pam50 = pam50.loc[common_patients]
clinical = clinical.loc[common_patients]


targets = pam50['BRCA_Subtype_PAM50'] 

print("\nFinal shapes:")
print(f"meth: {meth.shape}")
print(f"rna: {rna.shape}")
print(f"mirna: {mirna.shape}")
print(f"pam50: {pam50.shape}")
print(f"clinical: {clinical.shape}")
print(f"targets: {targets.shape}")

In [ ]:
# drop unwanted columns from clinical data
clinical.drop(columns=["Composite Element REF"], errors="ignore", inplace=True)

# we transform the methylation beta values to M-values and drop unwanted columns
meth_m = meth.drop(columns=["Composite Element REF"], errors="ignore")

# convert beta values to M-values using bioneuralnet utility with small epsilon to avoid log(0)
meth_m = bnn.utils.beta_to_m(meth_m, eps=1e-6) 

# lastly we turn the target labels into numerical classes
mapping_brca = {
    'LumA': 0, 
    'Her2': 1, 
    'LumB': 2, 
    'Basal': 3, 
    'Normal': 4
}
target_labels = targets.map(mapping_brca).to_frame(name="target")

X_meth = meth_m.loc[targets.index]
X_rna = rna.loc[targets.index]
X_mirna = mirna.loc[targets.index]
Y_labels = target_labels.loc[targets.index]
clinical_processed = clinical.loc[targets.index]


In [ ]:
display(X_meth.iloc[:3,:5])
display(X_meth.shape)

display(X_rna.iloc[:3,:5])
display(X_rna.shape)

display(X_mirna.iloc[:3,:5])
display(X_mirna.shape)

display(clinical_processed.iloc[:3,:5])
display(clinical_processed.shape)

display(Y_labels.value_counts())


## Feature Selection Methodology for BRCA

### Supported Methods and Interpretation

BioNeuralNet provides three techniques for feature selection, allowing for different views of the data's statistical profile:

* Variance Thresholding: Identifies features with the highest overall variance across all samples.

* ANOVA F-test: Pinpoints features that best distinguish between the target classes (LumA, LumB, Her2, Basal, and Normal).

    Random Forest Importance: Assesses feature utility based on its contribution to a predictive non-linear model.

### BRCA Cohort Selection Strategy

A dimensionality reduction step was essential for managing the high-feature-count omics data, given the complexity of the BRCA network:

* High-Feature Datasets: Both DNA Methylation (20,106 features) and RNA (18,321 features) required significant feature reduction.

* Filtering Process: The top 6,000 features were initially extracted from the Methylation and RNA datasets using all three methods.

* Final Set: A consensus set was built by finding the intersection of features selected by the ANOVA F-test and Random Forest Importance, ensuring both statistical relevance and model-based utility.

* Low-Feature Datasets: The miRNA data (503 features) was passed through without selection, as its feature count was already manageable.

In [ ]:
import bioneuralnet as bnn

# feature selection
meth_highvar = bnn.utils.select_top_k_variance(X_meth, k=6000)
rna_highvar = bnn.utils.select_top_k_variance(X_rna, k=6000)

meth_af = bnn.utils.top_anova_f_features(X_meth, Y_labels, max_features=6000)
rna_af = bnn.utils.top_anova_f_features(X_rna, Y_labels, max_features=6000)

meth_rf = bnn.utils.select_top_randomforest(X_meth, Y_labels, top_k=6000)
rna_rf = bnn.utils.select_top_randomforest(X_rna, Y_labels, top_k=6000)

meth_var_set = set(meth_highvar.columns)
meth_anova_set = set(meth_af.columns)
meth_rf_set = set(meth_rf.columns)

rna_var_set = set(rna_highvar.columns)
rna_anova_set = set(rna_af.columns)
rna_rf_set = set(rna_rf.columns)

meth_inter1 = list(meth_anova_set & meth_var_set)
meth_inter2 = list(meth_rf_set & meth_var_set)
meth_inter3 = list(meth_anova_set & meth_rf_set)
meth_all_three = list(meth_anova_set & meth_var_set & meth_rf_set)

rna_inter4 = list(rna_anova_set & rna_var_set)
rna_inter5 = list(rna_rf_set & rna_var_set)
rna_inter6 = list(rna_anova_set & rna_rf_set)
rna_all_three = list(rna_anova_set & rna_var_set & rna_rf_set)

In [ ]:
print("FROM THE 6000 Methylation feature selection:\n")
print(f"Anova-F & variance selection share: {len(meth_inter1)} features")
print(f"Random Forest & variance selection share: {len(meth_inter2)} features")
print(f"Anova-F & Random Forest share: {len(meth_inter3)} features")
print(f"All three methods agree on: {len(meth_all_three)} features")

In [ ]:
print("\nFROM THE 6000 RNA feature selection:\n")
print(f"Anova-F & variance selection share: {len(rna_inter4)} features")
print(f"Random Forest & variance selection share: {len(rna_inter5)} features")
print(f"Anova-F & Random Forest share: {len(rna_inter6)} features")
print(f"All three methods agree on: {len(rna_all_three)} features")

## Feature Selection Summary: ANOVA-RF Intersection

The final set of features was determined by the **intersection** of those highlighted by the **ANOVA F-test** and **Random Forest Importance**. This methodology provides a balanced filter, capturing features with both high class-separability (ANOVA) and significant predictive value in a non-linear model (Random Forest). The resulting feature pool is considered highly relevant for the subsequent modeling tasks.

### Feature Overlap Results

The table below quantifies the shared features identified by the different selection techniques for each omics type.

| Omics Data Type | ANOVA-F & Variance | RF & Variance | ANOVA-F & Random Forest (Selected) | All Three Agree |
| :--- | :--- | :--- | :--- | :--- |
| **Methylation** | 2,092 features | 1,870 features | **2,203 features** | 814 features |
| **RNA** | 2,359 features | 2,191 features | **2,500 features** | 1,124 features |

In [ ]:
X_meth_selected = X_meth[meth_inter3]
X_rna_selected = X_rna[rna_inter6]

print("\nFinal Shapes for Modeling")
print(f"Methylation (X1): {X_meth_selected.shape}")
print(f"RNA-Seq (X2): {X_rna_selected.shape}")
print(f"miRNA-Seq (X3): {X_mirna.shape}")
print(f"Labels (Y): {Y_labels.shape}")

## Data Availability

To facilitate rapid experimentation and reproduction of our results, the fully processed and feature-selected dataset used in this analysis has been made available directly within the package.

Users can load this dataset, bypassing all preceding data acquisition, preprocessing, and feature selection steps. This allows users to proceed immediately from this step.

In [ ]:
from bioneuralnet.datasets import DatasetLoader

tgca_brca = DatasetLoader("brca")
display(tgca_brca.shape)

dna_meth = tgca_brca["meth"]
rna = tgca_brca["rna"]
mirna = tgca_brca["mirna"]
clinical = tgca_brca["clinical"]
target = tgca_brca["target"]

display(dna_meth.iloc[:3,:5])
display(rna.iloc[:3,:5])
display(mirna.iloc[:3,:5])
display(clinical.iloc[:3,:5])
display(target.iloc[:3,:5])

In [ ]:
import pandas as pd
import bioneuralnet as bnn

# subset from dna_methylation and rna to reduce dimensionality from 5000+ to 2525
dna_meth = bnn.utils.select_top_k_variance(dna_meth, k=1011)
rna = bnn.utils.select_top_k_variance(rna, k=1011)

## Reproducibility and Seeding

To ensure our experimental results are fully reproducible, a single global seed is set at the beginning of the analysis.

This utility function propagates the seed to all sources of randomness, including `random`, `numpy`, and `torch` (for both CPU and GPU). Critically, it also configures the PyTorch cuDNN backend to use deterministic algorithms.

In [ ]:
import bioneuralnet as bnn

SEED = 8183
bnn.utils.set_seed(SEED)

In [ ]:
# Identified several columns in the clinical data that could lead to data leakage
dataleak_columns = [
    'treatments_pharmaceutical_days_to_treatment_end',
    'treatments_pharmaceutical_days_to_treatment_start',
    'treatments_pharmaceutical_treatment_id',
    'treatments_pharmaceutical_treatment_type',
    'treatments_pharmaceutical_regimen_or_line_of_therapy',
    'treatments_pharmaceutical_treatment_effect',
    'treatments_pharmaceutical_therapeutic_agents',
    'treatments_pharmaceutical_treatment_or_therapy',
    'treatments_pharmaceutical_initial_disease_status',
    'treatments_pharmaceutical_treatment_intent_type',
    'treatments_pharmaceutical_treatment_anatomic_site',
    'treatments_pharmaceutical_treatment_outcome',
    'treatments_pharmaceutical_prescribed_dose_units',
    'treatments_pharmaceutical_treatment_dose',
    'treatments_pharmaceutical_route_of_administration',
    'treatments_pharmaceutical_number_of_cycles',
    'treatments_pharmaceutical_prescribed_dose',
    'treatments_pharmaceutical_treatment_dose_units',
    'treatments_pharmaceutical_clinical_trial_indicator',
    'treatments_pharmaceutical_treatment_anatomic_sites',
    'treatments_pharmaceutical_margin_status',
    'treatments_pharmaceutical_course_number',
    'treatments_pharmaceutical_number_of_fractions',
    'treatments_radiation_days_to_treatment_end',
    'treatments_radiation_days_to_treatment_start',
    'treatments_radiation_treatment_id',
    'treatments_radiation_treatment_type',
    'treatments_radiation_regimen_or_line_of_therapy',
    'treatments_radiation_treatment_effect',
    'treatments_radiation_therapeutic_agents',
    'treatments_radiation_treatment_or_therapy',
    'treatments_radiation_initial_disease_status',
    'treatments_radiation_treatment_intent_type',
    'treatments_radiation_treatment_anatomic_site',
    'treatments_radiation_treatment_outcome',
    'treatments_radiation_prescribed_dose_units',
    'treatments_radiation_treatment_dose',
    'treatments_radiation_route_of_administration',
    'treatments_radiation_number_of_cycles',
    'treatments_radiation_prescribed_dose',
    'treatments_radiation_treatment_dose_units',
    'treatments_radiation_clinical_trial_indicator',
    'treatments_radiation_treatment_anatomic_sites',
    'treatments_radiation_margin_status',
    'treatments_radiation_course_number',
    'treatments_radiation_number_of_fractions',
    'radiation_therapy',
    'patient',
    'ajcc_pathologic_stage',
    'ajcc_pathologic_t',
    'ajcc_pathologic_n',
    'ajcc_pathologic_m',
    'age_at_diagnosis',
    'primary_diagnosis',
   'tissue_or_organ_of_origin',
    'days_to_last_follow_up',
    'year_of_diagnosis',
    
    'vital_status',
    'days_to_death',
    'year_of_death',
    'days_to_last_follow_up',
    'days_to_last_known_alive',
    'last_known_disease_status',
    'days_to_last_known_disease_status',
    'days_to_recurrence',
    'progression_or_recurrence',
    'age_at_diagnosis',
    'days_to_birth',  
]

clinical_clean = clinical.drop(columns=dataleak_columns, errors='ignore')

In [ ]:
from bioneuralnet.utils import preprocess_clinical

clinical_for_model = preprocess_clinical(
    clinical_clean, 
    top_k=7, 
    scale=False,
)

print(clinical_for_model.columns)
display(clinical_for_model.iloc[:5,:5])

In [ ]:
from bioneuralnet.utils import gen_threshold_graph, gen_correlation_graph, gen_similarity_graph, gen_gaussian_knn_graph

omics_brca = pd.concat([dna_meth, rna, mirna], axis=1)

threshold_10 = gen_threshold_graph(omics_brca, b=6.2, k=10)
correlation_10 = gen_correlation_graph(omics_brca, k=10, method='pearson', signed=False)
similarity_10 = gen_similarity_graph(omics_brca, k=10, metric='cosine')
gaussian_15 = gen_gaussian_knn_graph(omics_brca, k=15, sigma=None)

comparison_runs = [
    {"name":"threshold_10", "graph": threshold_10},
    {"name":"correlation_10", "graph": correlation_10},
    {"name":"similarity_10", "graph": similarity_10},
    {"name":"gaussian_15", "graph": gaussian_15},
]

## Classification using DPMON: Training and Evaluation

In [ ]:
from bioneuralnet.downstream_task import DPMON


output_dir_base = Path("/home/vicente/Github/BioNeuralNet/dpmon_results_GAT_FINAL/brca")
all_results = []

for run_config in comparison_runs:
    graph_name = run_config["name"]
    A_full = run_config["graph"]
    
    current_output_dir = output_dir_base / graph_name
    current_output_dir.mkdir(parents=True, exist_ok=True)

    dpmon_params_base = {
        "adjacency_matrix": A_full,
        "omics_list": omics_brca,
        "phenotype_data": target,
        "phenotype_col": "target",
        "clinical_data": clinical_for_model,
        "model": 'GAT',
        "tune": True, 
        "cv": True,   
        "n_folds": 5,
        "repeat_num":5,
        "gpu": True,
        "cuda": 0,
        "seed": SEED,
        "output_dir": current_output_dir
    }
    
    dpmon_tunned = DPMON(**dpmon_params_base)
    predictions_df, metrics, embeddings = dpmon_tunned.run()

    all_results.append({
        "graph": graph_name,
        "predictions": predictions_df,
        "metrics": metrics,
        "embeddings": embeddings
    })

for res in all_results:
    graph_name = res["graph"]
    graph_metrics = res["metrics"]
    
    acc_row = graph_metrics.loc[graph_metrics['Metric'] == 'Accuracy'].iloc[0]
    f1_macro_row = graph_metrics.loc[graph_metrics['Metric'] == 'F1 Macro'].iloc[0]
    f1_weighted_row = graph_metrics.loc[graph_metrics['Metric'] == 'F1 Weighted'].iloc[0]
    recall_row = graph_metrics.loc[graph_metrics['Metric'] == 'Recall'].iloc[0]
    auc_row = graph_metrics.loc[graph_metrics['Metric'] == 'AUC'].iloc[0]
    
    acc_avg, acc_std = acc_row['Average'], acc_row['StdDev']
    f1_macro_avg, f1_macro_std = f1_macro_row['Average'], f1_macro_row['StdDev']
    f1_weighted_avg, f1_weighted_std = f1_weighted_row['Average'], f1_weighted_row['StdDev']
    recall_avg, recall_std = recall_row['Average'], recall_row['StdDev']
    auc_avg, auc_std = auc_row['Average'], auc_row['StdDev']

    print(f"\nResults for: {graph_name}")
    print(f"Accuracy (Avg +/- Std): {acc_avg:.4f} +/- {acc_std:.4f}")
    print(f"F1 Macro (Avg +/- Std): {f1_macro_avg:.4f}  +/- {f1_macro_std:.4f}")
    print(f"F1 Weighted (Avg +/- Std): {f1_weighted_avg:.4f} +/- {f1_weighted_std:.4f}")
    print(f"Recall: {recall_avg:.4f} +/- {recall_std:.4f}")
    print(f"AUC: {auc_avg:.4f} +/- {auc_std:.4f}")

In [ ]:
output_dir_base = Path("/home/vicente/Github/BioNeuralNet/dpmon_results_SAGE_FINAL/brca")
all_results = []

for run_config in comparison_runs:
    graph_name = run_config["name"]
    A_full = run_config["graph"]
    
    current_output_dir = output_dir_base / graph_name
    current_output_dir.mkdir(parents=True, exist_ok=True)

    dpmon_params_base = {
        "adjacency_matrix": A_full,
        "omics_list": omics_brca,
        "phenotype_data": target,
        "phenotype_col": "target",
        "clinical_data": clinical_for_model,
        "model": 'SAGE',
        "tune": True, 
        "cv": True,   
        "n_folds": 5,
        "repeat_num":5,
        "gpu": True,
        "cuda": 0,
        "seed": SEED,
        "output_dir": current_output_dir
    }
    
    dpmon_tunned = DPMON(**dpmon_params_base)
    predictions_df, metrics, embeddings = dpmon_tunned.run()

    all_results.append({
        "graph": graph_name,
        "predictions": predictions_df,
        "metrics": metrics,
        "embeddings": embeddings
    })

for res in all_results:
    graph_name = res["graph"]
    graph_metrics = res["metrics"]
    
    acc_row = graph_metrics.loc[graph_metrics['Metric'] == 'Accuracy'].iloc[0]
    f1_macro_row = graph_metrics.loc[graph_metrics['Metric'] == 'F1 Macro'].iloc[0]
    f1_weighted_row = graph_metrics.loc[graph_metrics['Metric'] == 'F1 Weighted'].iloc[0]
    recall_row = graph_metrics.loc[graph_metrics['Metric'] == 'Recall'].iloc[0]
    auc_row = graph_metrics.loc[graph_metrics['Metric'] == 'AUC'].iloc[0]
    
    acc_avg, acc_std = acc_row['Average'], acc_row['StdDev']
    f1_macro_avg, f1_macro_std = f1_macro_row['Average'], f1_macro_row['StdDev']
    f1_weighted_avg, f1_weighted_std = f1_weighted_row['Average'], f1_weighted_row['StdDev']
    recall_avg, recall_std = recall_row['Average'], recall_row['StdDev']
    auc_avg, auc_std = auc_row['Average'], auc_row['StdDev']

    print(f"\nResults for: {graph_name}")
    print(f"Accuracy (Avg +/- Std): {acc_avg:.4f} +/- {acc_std:.4f}")
    print(f"F1 Macro (Avg +/- Std): {f1_macro_avg:.4f}  +/- {f1_macro_std:.4f}")
    print(f"F1 Weighted (Avg +/- Std): {f1_weighted_avg:.4f} +/- {f1_weighted_std:.4f}")
    print(f"Recall: {recall_avg:.4f} +/- {recall_std:.4f}")
    print(f"AUC: {auc_avg:.4f} +/- {auc_std:.4f}")

In [ ]:
output_dir_base = Path("/home/vicente/Github/BioNeuralNet/dpmon_results_GCN_FINAL/brca")

all_results = []

for run_config in comparison_runs:
    graph_name = run_config["name"]
    A_full = run_config["graph"]
    
    current_output_dir = output_dir_base / graph_name
    current_output_dir.mkdir(parents=True, exist_ok=True)

    dpmon_params_base = {
        "adjacency_matrix": A_full,
        "omics_list": omics_brca,
        "phenotype_data": target,
        "phenotype_col": "target",
        "clinical_data": clinical_for_model,
        "model": 'GCN',
        "tune": True, 
        "cv": True,   
        "n_folds": 5,
        "repeat_num":5,
        "gpu": True,
        "cuda": 0,
        "seed": SEED,
        "output_dir": current_output_dir
    }
    
    dpmon_tunned = DPMON(**dpmon_params_base)
    predictions_df, metrics, embeddings = dpmon_tunned.run()

    all_results.append({
        "graph": graph_name,
        "predictions": predictions_df,
        "metrics": metrics,
        "embeddings": embeddings
    })

for res in all_results:
    graph_name = res["graph"]
    graph_metrics = res["metrics"]
    
    acc_row = graph_metrics.loc[graph_metrics['Metric'] == 'Accuracy'].iloc[0]
    f1_macro_row = graph_metrics.loc[graph_metrics['Metric'] == 'F1 Macro'].iloc[0]
    f1_weighted_row = graph_metrics.loc[graph_metrics['Metric'] == 'F1 Weighted'].iloc[0]
    recall_row = graph_metrics.loc[graph_metrics['Metric'] == 'Recall'].iloc[0]
    auc_row = graph_metrics.loc[graph_metrics['Metric'] == 'AUC'].iloc[0]
    
    acc_avg, acc_std = acc_row['Average'], acc_row['StdDev']
    f1_macro_avg, f1_macro_std = f1_macro_row['Average'], f1_macro_row['StdDev']
    f1_weighted_avg, f1_weighted_std = f1_weighted_row['Average'], f1_weighted_row['StdDev']
    recall_avg, recall_std = recall_row['Average'], recall_row['StdDev']
    auc_avg, auc_std = auc_row['Average'], auc_row['StdDev']

    print(f"\nResults for: {graph_name}")
    print(f"Accuracy (Avg +/- Std): {acc_avg:.4f} +/- {acc_std:.4f}")
    print(f"F1 Macro (Avg +/- Std): {f1_macro_avg:.4f}  +/- {f1_macro_std:.4f}")
    print(f"F1 Weighted (Avg +/- Std): {f1_weighted_avg:.4f} +/- {f1_weighted_std:.4f}")
    print(f"Recall: {recall_avg:.4f} +/- {recall_std:.4f}")
    print(f"AUC: {auc_avg:.4f} +/- {auc_std:.4f}")

In [ ]:
from sklearn.model_selection import StratifiedKFold, ParameterSampler, RepeatedStratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score, average_precision_score
from sklearn.base import clone
from sklearn.preprocessing import StandardScaler, label_binarize
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from scipy.stats import loguniform, randint
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
import numpy as np
import pandas as pd

X = pd.concat([dna_meth, rna, mirna, clinical_for_model], axis=1)
y = target['vital_status']
print(f"Successfully created X matrix with shape: {X.shape}")
print(f"Successfully created y vector with shape: {y.shape}")


pipe_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LogisticRegression(solver='lbfgs', max_iter=1000, penalty=None, random_state=SEED))
])

pipe_mlp = Pipeline([
    ('scaler', StandardScaler()),
    ('model', MLPClassifier(max_iter=500, early_stopping=True, n_iter_no_change=10, random_state=SEED))
])

pipe_xgb = Pipeline([
    ('scaler', StandardScaler()),
    ('model', XGBClassifier(eval_metric='logloss', tree_method='hist', max_bin=128, random_state=SEED))
])
pipe_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RandomForestClassifier(random_state=SEED))
])

pipe_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('model', SVC(probability=True, random_state=SEED))
])

pipe_dt = Pipeline([
    ('scaler', StandardScaler()),
    ('model', DecisionTreeClassifier(random_state=SEED))
])

params_lr = {'model__penalty': ['l2'], 'model__C': loguniform(1e-4, 1e2)}

params_mlp = {
    'model__hidden_layer_sizes': [(100,), (100, 50), (50, 50)],
    'model__activation': ['relu', 'tanh'],
    'model__alpha': loguniform(1e-5, 1e-1),
    'model__learning_rate_init': loguniform(1e-4, 1e-2)
}
params_xgb = {
    'model__n_estimators': randint(50, 200),
    'model__learning_rate': loguniform(0.01, 0.3),
    'model__max_depth': randint(3, 7),
    'model__subsample': [0.8, 1.0], 
    'model__colsample_bytree': [0.8, 1.0]
}
params_rf = {
    'model__n_estimators': randint(100, 300),
    'model__max_depth': [10, 20, 30, None],
    'model__min_samples_split': randint(2, 10),
    'model__min_samples_leaf': randint(1, 5),
    'model__max_features': ['sqrt', 'log2']
}
params_svm = {
    'model__C': loguniform(1e-2, 1e3),
    'model__kernel': ['rbf', 'linear'],
    'model__gamma': ['scale', 'auto']
}

params_dt = {
    'model__max_depth': randint(3, 15),
    'model__min_samples_split': randint(2, 20),
    'model__criterion': ['gini', 'entropy']
}

models_to_tune = {
    "LogisticRegression": (pipe_lr, params_lr),
    "SVM": (pipe_svm, params_svm),
    "MLP": (pipe_mlp, params_mlp),
    "XGBoost": (pipe_xgb, params_xgb),
    "RandomForest": (pipe_rf, params_rf),
    "DecisionTree": (pipe_dt, params_dt),
}

all_results = {
    "LogisticRegression": {"acc": [], "f1_w": [], "f1_m": [], "recall": [], "auc": []},
    "MLP": {"acc": [], "f1_w": [], "f1_m": [], "recall": [], "auc": []},
    "XGBoost": {"acc": [], "f1_w": [], "f1_m": [], "recall": [], "auc": []},
    "RandomForest": {"acc": [], "f1_w": [], "f1_m": [], "recall": [], "auc": []},
    "SVM": {"acc": [], "f1_w": [], "f1_m": [], "recall": [], "auc": []},
    "DecisionTree": {"acc": [], "f1_w": [], "f1_m": [], "recall": [], "auc": []},
}


for model_name, (pipeline, param_dist) in models_to_tune.items():
    print(f"Evaluating model with nested CV: {model_name}")
    
    outer_cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=SEED)
    
    # inner folds are for finding the best hyperparameters
    for fold_idx, (train_idx, test_idx) in enumerate(outer_cv.split(X, y), start=1):
        X_train_outer, X_test_outer = X.iloc[train_idx], X.iloc[test_idx]
        y_train_outer, y_test_outer = y.iloc[train_idx], y.iloc[test_idx]
        inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=SEED)
        
        best_score_fold = -np.inf
        best_params_fold = None
        # May or may not want to set a seed here. A fix seed = same hyperparamters each fold.
        # No seed = different hyperparameters each fold. This adds more randomness, and may yield better generalization.
        param_sampler = list(ParameterSampler(param_dist, n_iter=20))
        
        for params in param_sampler:
            inner_scores = []
            
            for inner_train_idx, inner_val_idx in inner_cv.split(X_train_outer, y_train_outer):
                X_train_inner = X_train_outer.iloc[inner_train_idx]
                X_val_inner = X_train_outer.iloc[inner_val_idx]
                y_train_inner = y_train_outer.iloc[inner_train_idx]
                y_val_inner = y_train_outer.iloc[inner_val_idx]
                
                inner_pipeline = clone(pipeline)
                inner_pipeline.set_params(**params)
                inner_pipeline.fit(X_train_inner, y_train_inner)
                
                y_val_pred = inner_pipeline.predict(X_val_inner)
                score = f1_score(y_val_inner, y_val_pred, average='weighted', zero_division=0)
                inner_scores.append(score)
            
            mean_score = np.mean(inner_scores)
            if mean_score > best_score_fold:
                best_score_fold = mean_score
                best_params_fold = params
        
        print(f"Outer fold {fold_idx}: best params (inner CV F1-W={best_score_fold:.4f})")
        print(f"{best_params_fold}")
        
        final_pipeline = clone(pipeline)
        final_pipeline.set_params(**best_params_fold)
        final_pipeline.fit(X_train_outer, y_train_outer)
        
        preds = final_pipeline.predict(X_test_outer)
        
        if hasattr(final_pipeline, "predict_proba"):
            proba = final_pipeline.predict_proba(X_test_outer)
        else:
            proba = None
        
        acc = accuracy_score(y_test_outer, preds)
        f1_w = f1_score(y_test_outer, preds, average='weighted', zero_division=0)
        f1_m = f1_score(y_test_outer, preds, average='macro', zero_division=0)
        recall = recall_score(y_test_outer, preds, average='macro', zero_division=0)
        
        auc = np.nan
        
        if proba is not None:
            try:
                if len(np.unique(y)) == 2:
                    auc = roc_auc_score(y_test_outer, proba[:, 1])
                else:
                    auc = roc_auc_score(y_test_outer, proba, multi_class='ovr', average='macro')
                    y_test_bin = label_binarize(y_test_outer, classes=np.unique(y))
            except Exception:
                auc = np.nan

        print(f"Fold {fold_idx} results: Acc={acc:.4f}, F1-W={f1_w:.4f}, "
              f"F1-M={f1_m:.4f}, Recall={recall:.4f}, AUC={auc:.4f}")
        
        all_results[model_name]["acc"].append(acc)
        all_results[model_name]["f1_w"].append(f1_w)
        all_results[model_name]["f1_m"].append(f1_m)
        all_results[model_name]["recall"].append(recall)
        all_results[model_name]["auc"].append(auc)

print("\nFINAL BASELINE RESULTS\n")
for model_name, metrics in all_results.items():
    avg_acc = np.mean(metrics["acc"])
    std_acc = np.std(metrics["acc"])
    avg_f1_w = np.mean(metrics["f1_w"])
    std_f1_w = np.std(metrics["f1_w"])
    avg_f1_m = np.mean(metrics["f1_m"])
    std_f1_m = np.std(metrics["f1_m"])
    avg_recall = np.mean(metrics["recall"])
    std_recall = np.std(metrics["recall"])
    avg_auc = np.nanmean(metrics["auc"])
    std_auc = np.nanstd(metrics["auc"])
    
    print(f"\n{model_name}:")
    print(f"Accuracy: {avg_acc:.4f} +/- {std_acc:.4f}")
    print(f"F1 Weighted: {avg_f1_w:.4f} +/- {std_f1_w:.4f}")
    print(f"F1 Macro: {avg_f1_m:.4f} +/- {std_f1_m:.4f}")
    print(f"Recall: {avg_recall:.4f} +/- {std_recall:.4f}")
    print(f"AUC: {avg_auc:.4f} +/- {std_auc:.4f}")